In [1]:
from sklearn.preprocessing import StandardScaler
import os
import pandas as pd
from sklearn import datasets
import plotly as plt
import plotly.express as px
from features import *
from clustering import *
from utils import *
from constant import  PATH_OUTPUT, MODEL_CLUSTERING, PATH_DATA, PATH_DATA_ALL

from sklearn.cluster import KMeans

from sklearn.pipeline import Pipeline

from skimage.transform import resize

In [2]:
from images import load_images_from_folder

# Example usage:
folder_path = PATH_DATA_ALL + "/code_test"
images, labels_true, folder_names, smallest_height, smallest_width, smallest_height2, smallest_width2 = load_images_from_folder(folder_path)
taille = len(images)
nombre_de_canaux = 3
# print(f"Smallest height: {smallest_height}")
# print(f"Smallest width: {smallest_width}")
# print(f"Hightest height: {smallest_height2}")
# print(f"Hightest width: {smallest_width2}")

In [3]:

# Supposons que `images` est votre liste d'images en RGB
descriptors_hsv = convert_color_space(images, "HSV") # ou "Lab"

In [4]:
# descriptors_sift = compute_sift_descriptors(images)
sift_descriptors = extract_sift_features(images)

# Étape 2 : Création des vecteurs de caractéristiques avec Bag of Features
descriptors_sift = create_bag_of_features(sift_descriptors, n_clusters=50)

c:\Users\comma\Documents\travail\Polytech\s8\apprentissage_automatique\projet\sujet_tp\sujet_tp\src\features.py:127: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  features = np.array(


In [5]:
images_to_use = descriptors_hsv

In [6]:

descriptors_hog = compute_hog_descriptors(images_to_use)

In [7]:
descriptors_hist = compute_gray_histograms(images_to_use)

In [8]:
print(np.array(descriptors_hog).shape)
print(np.array(descriptors_hsv).shape)
print(np.array(descriptors_sift).shape)

(42, 8192)
(42, 256, 256, 3)
(42, 50)


**3. Application de RBM et KMEANS**

In [9]:
from sklearn.preprocessing import StandardScaler

descriptors = descriptors_sift

# Présumons que `images` est votre liste d'images prétraitées et aplatie en vecteurs

# Initialisation de la classe StackedRBM
stacked_rbm = StackedRBM(n_components_list=[256, 128], n_iter=10, learning_rate=0.01, batch_size=10)

# Ajustement des RBMs sur les données d'image
stacked_rbm.fit(descriptors)

# Transformation des images en nouvelles représentations avec les RBMs entraînés
transformed_images = stacked_rbm.transform(descriptors)

# Normalisation des caractéristiques pour améliorer les performances de K-Means
scaler = StandardScaler()
transformed_images_scaled = scaler.fit_transform(transformed_images)

# Clustering avec K-Means
kmeans = KMeans(n_clusters=20, random_state=42)
clusters = kmeans.fit_predict(transformed_images_scaled)

[BernoulliRBM] Iteration 1, pseudo-likelihood = -327.08, time = 0.00s
[BernoulliRBM] Iteration 2, pseudo-likelihood = -644.49, time = 0.01s
[BernoulliRBM] Iteration 3, pseudo-likelihood = -569.83, time = 0.01s
[BernoulliRBM] Iteration 4, pseudo-likelihood = -2253.87, time = 0.01s
[BernoulliRBM] Iteration 5, pseudo-likelihood = -1211.82, time = 0.01s
[BernoulliRBM] Iteration 6, pseudo-likelihood = -2495.81, time = 0.01s
[BernoulliRBM] Iteration 7, pseudo-likelihood = -3655.63, time = 0.01s
[BernoulliRBM] Iteration 8, pseudo-likelihood = -2583.74, time = 0.01s
[BernoulliRBM] Iteration 9, pseudo-likelihood = -4422.98, time = 0.01s
[BernoulliRBM] Iteration 10, pseudo-likelihood = -3620.09, time = 0.01s
[BernoulliRBM] Iteration 1, pseudo-likelihood = -184.41, time = 0.01s
[BernoulliRBM] Iteration 2, pseudo-likelihood = -178.74, time = 0.01s
[BernoulliRBM] Iteration 3, pseudo-likelihood = -176.35, time = 0.01s
[BernoulliRBM] Iteration 4, pseudo-likelihood = -174.94, time = 0.01s
[BernoulliRB

c:\Users\comma\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:1474: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (20). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


In [10]:
from sklearn.metrics import silhouette_score

# Calcul du Silhouette Score
score = silhouette_score(transformed_images_scaled, clusters)

print("Silhouette Score:", score)

Silhouette Score: 1.0
